# Simple Computational Graph (Implementation and Examples)
The following code is a simple implementation of the forward pass for a computational graph.

Farhad Kamangar  Oct. 2019

In [1]:
import numpy as np
class Node:
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.consumers = []
        for input_node in input_nodes:
            input_node.consumers.append(self)
        global_default_graph.operations.append(self)
    def compute(self):
        pass
class add(Node):
    def __init__(self, x, y):
        super().__init__([x, y])
    def compute(self, x_value, y_value):
        return x_value + y_value
class matmul(Node):
    def __init__(self, x, y):
        super().__init__([x, y])
    def compute(self, x_value, y_value):
        return x_value.dot(y_value)
class sigmoid(Node):
    def __init__(self, x):
        super().__init__([x])
    def compute(self, x_value):
        return 1/(1+np.exp(-x_value))
class placeholder:
    def __init__(self):
        self.consumers = []
        global_default_graph.placeholders.append(self)
class Variable:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.consumers = []
        global_default_graph.variables.append(self)
class Graph:
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = [1,2,3]
    def create_default_graph(self):
        global global_default_graph
        global_default_graph = self
class Session:
    def run(self, target_node, feed_dict={}):
        postorder_nodes_list = postorder(target_node)
        for node in postorder_nodes_list:
            if type(node) == placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs=[]
                for input_node in node.input_nodes:
                    node.inputs.append(input_node.output)
                node.output = node.compute(*node.inputs)
            if type(node.output) == list:
                node.output = np.array(node.output)
        return target_node.output
def postorder(operation):
    postorder_nodes_list = []
    def recursive_traverse(node):
        if isinstance(node, Node):
            for input_node in node.input_nodes:
                recursive_traverse(input_node)
        postorder_nodes_list.append(node)
    recursive_traverse(operation)
    return postorder_nodes_list

## Example #1
The following code creates a computational graph for the equation $y=ax+b$

In [2]:
# Example #1: y=a*b+c 

Graph().create_default_graph()  # Create a new graph
a=Variable([3])
x = Variable([5])
c = Variable([8])
ax=matmul(a,x)  # ax is the temp name of the matmul node
y=add(ax,c)
session = Session()
output = session.run(y)
print("y=",output)

y= [23]


## Example #2
The following code creates a computational graph for the equation $Y=AX+B$

The difference between this example and the previous one is that this example works with numpy arrays
Note:
A and B are variables
AX is an intermediate node

In [3]:
A=Variable(np.array([[1, 2], [5, 3]]))
X=Variable(np.array([[1,5,7,5,2],[2,6,9,3,8]]))
B = Variable(np.array([[5],[4]]))
AX = matmul(A, X)
Y = add(AX,B)
session = Session()
output = session.run(Y)
print("A=",A.value)
print("B=",B.value)
print("X=",X.output)
print("Y=",Y.output)

A= [[1 2]
 [5 3]]
B= [[5]
 [4]]
X= [[1 5 7 5 2]
 [2 6 9 3 8]]
Y= [[10 22 30 16 23]
 [15 47 66 38 38]]


## Example #3
The following code creates a computational graph for the equation $Y=AX+B$

The difference between this example and the previous one is that X is defined as a place holder.
A placeholder is a variable that we will assign data to it when we run the session. It allows us to create the computation graph without needing the data.
Note:
A and B are variables
AX is an intermediate node

In [4]:
A=Variable(np.array([[1, 2], [5, 3]]))
X = placeholder()
B = Variable(np.array([[5],[4]]))
AX = matmul(A, X)
Y = add(AX,B)
session = Session()
output = session.run(Y, {X: np.array([[1,5,7,5,2],[2,6,9,3,8]])})
print("A=",A.value)
print("B=",B.value)
print("X=",X.output)
print("Y=",Y.output)

A= [[1 2]
 [5 3]]
B= [[5]
 [4]]
X= [[1 5 7 5 2]
 [2 6 9 3 8]]
Y= [[10 22 30 16 23]
 [15 47 66 38 38]]


## Example #4
The following code creates a computational graph for a single neuron with sigmoid activation.



In [5]:
# Example #4: single neuron with sigmoid activation
weights=Variable(np.random.randn(1,2))
bias=Variable(np.random.randn(1,1))
X = placeholder()
WX = matmul(weights,X)
net=add(WX,bias)
a=sigmoid(net)

session = Session()
output = session.run(a, {X: np.array([[1,5,7,5,2],[2,6,9,3,8]])})
print("weights=",weights.value)
print("bias=",bias.value)
print("X=",X.output)
print("output=",output)

weights= [[ 0.02254587 -0.83461608]]
bias= [[0.46545614]]
X= [[1 5 7 5 2]
 [2 6 9 3 8]]
output= [[0.23483108 0.0117799  0.00101864 0.12723143 0.00209442]]


## Example #4
The following code creates a computational graph for a single layer of neurons with sigmoid activation.


In [6]:
# Example #5: One layer with multiple neurons with sigmoid activation
input_dimensions=10
number_of_neurons=5
number_of_samples=3
weights=Variable(np.random.randn(number_of_neurons,input_dimensions))
bias=Variable(np.random.randn(number_of_neurons,1))
X = placeholder()
WX = matmul(weights,X)
net=add(WX,bias)
a=sigmoid(net)

session = Session()
output = session.run(a, {X: np.random.randn(input_dimensions,number_of_samples)})
print("weights=",weights.value)
print("bias=",bias.value)
print("X=",X.output)
print("output=",output)


weights= [[ 1.05388052e+00  1.56509813e+00  4.91534235e-01  1.06411649e+00
   4.85007033e-01 -1.23921998e+00  5.50302413e-01 -4.52099888e-01
   1.29024265e+00  1.53990063e+00]
 [-2.94724443e+00  6.83339418e-01 -5.04842489e-01 -1.50325680e+00
  -1.47939769e-01 -3.40681532e-01 -6.02712826e-01  1.09762365e+00
  -4.43516416e-01 -8.68603601e-02]
 [-9.68173272e-01 -7.75999293e-01  1.87545466e+00  7.25124028e-02
  -2.75314387e-01  2.52486735e+00 -7.45182470e-01 -1.21869801e+00
  -5.10034816e-01  2.37350183e+00]
 [ 8.37077432e-01 -5.68456521e-02 -1.85735192e+00 -7.23362678e-01
  -2.12724479e-01 -2.58164057e+00  8.45586048e-01  1.53971428e-03
   4.18172397e-01  1.61802068e+00]
 [ 5.46629350e-01 -8.14861057e-01 -5.69135807e-01  1.64763550e+00
  -5.48009929e-01 -8.28541942e-01 -5.83146345e-01 -7.96672155e-01
   1.02237622e-02  3.53976780e-02]]
bias= [[-0.78657798]
 [ 0.06280816]
 [-0.71284611]
 [ 1.5157049 ]
 [ 0.98210131]]
X= [[-0.77638989  0.45450213  0.90849114]
 [-0.24181127 -0.27384975 -1.79